In [1]:
#####Symmetric Correspondence LDA#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

C:\Users\sana\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [3]:
####データの発生####
##データの設定
L = 3   #データ数
k = 15   #トピック数
d = 3000   #文書数
v1 = 1000; v2 = 800; v3 = 700   
v = [v1, v2, v3]   #語彙数
w1 = np.random.poisson(np.random.gamma(85, 1/0.6, d), d)
w2 = np.random.poisson(np.random.gamma(80, 1/0.65, d), d)
w3 = np.random.poisson(np.random.gamma(75, 1/0.65, d), d)
w = [w1, w2, w3]   #単語数
f1 = np.sum(w1); f2 = np.sum(w2); f3 = np.sum(w3)   #総単語数
f = np.array([f1, f2, f3])

In [4]:
##IDとインデックスの設定
#IDの設定
d_id1 = np.repeat(range(d), w1)
d_id2 = np.repeat(range(d), w2)
d_id3 = np.repeat(range(d), w3)

#インデックスの設定
d_list1 = [i for i in range(d)]; d_vec1 = [i for i in range(d)]
d_list2 = [i for i in range(d)]; d_vec2 = [i for i in range(d)]
d_list3 = [i for i in range(d)]; d_vec3 = [i for i in range(d)]
for i in range(d):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int")
    d_vec1[i] = np.repeat(1, w1[i])
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int")
    d_vec2[i] = np.repeat(1, w2[i])
    d_list3[i] = np.array(np.where(d_id3==i)[0], dtype="int")
    d_vec3[i] = np.repeat(1, w3[i])
d_list = [d_list1, d_list2, d_list3]
d_vec = [d_vec1, d_vec2, d_vec3]

#スパース行列に変換
d_dt1 = sparse.coo_matrix((np.repeat(1, f1), (d_id1, np.arange(f1))), shape=(d, f1)).tocsr()
d_dt2 = sparse.coo_matrix((np.repeat(1, f2), (d_id2, np.arange(f2))), shape=(d, f2)).tocsr()
d_dt3 = sparse.coo_matrix((np.repeat(1, f3), (d_id3, np.arange(f3))), shape=(d, f3)).tocsr()
d_dt = [d_dt1, d_dt2, d_dt3]

In [5]:
##データの生成
rp = 0
while True:
    rp = rp + 1
    print(rp)

    ##パラメータの生成
    #ディリクレ分布のパラメータ
    alpha1 = np.repeat(0.25, L)
    alpha2 = np.repeat(0.1, k)
    beta1 = np.repeat(0.05, v1)
    beta2 = np.repeat(0.05, v2)
    beta3 = np.repeat(0.05, v3)

    #ディリクレ分布からトピック分布を生成
    theta1 = np.random.dirichlet(alpha1, d)
    theta2 = np.zeros((d, k, L))
    for j in range(L):
        theta2[:, :, j] = np.random.dirichlet(alpha2, d)
    thetat1 = theta1.copy(); thetat2 = theta2.copy()

    #ディリクレ分布から単語分布を生成
    phi1 = np.random.dirichlet(beta1, k)  
    phi2 = np.random.dirichlet(beta2, k)  
    phi3 = np.random.dirichlet(beta3, k)  

    #出現確率が低い単語を入れ替える
    index_v1 = np.array(range(v1))[np.max(phi1, axis=0) <= (k*5)/f1]
    for j in range(index_v1.shape[0]):
        phi1[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v1[j]] = (k*5)/f1
    phi1 = phi1 / np.sum(phi1, axis=1).reshape(k, 1)
    phit1 = phi1.copy()

    index_v2 = np.array(range(v2))[np.max(phi2, axis=0) <= (k*5)/f2]
    for j in range(index_v2.shape[0]):
        phi2[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v2[j]] = (k*5)/f1
    phi2 = phi2 / np.sum(phi2, axis=1).reshape(k, 1)
    phit2 = phi2.copy()

    index_v3 = np.array(range(v3))[np.max(phi3, axis=0) <= (k*5)/f3]
    for j in range(index_v3.shape[0]):
        phi3[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v3[j]] = (k*5)/f3
    phi3 = phi3 / np.sum(phi3, axis=1).reshape(k, 1)
    phit3 = phi3.copy()

    #単語分布をリストに格納
    phi = [phi1, phi2, phi3]
    phit = phi.copy()

    
    ##トピックと単語を生成
    #モードごとのオブジェクトの格納用リスト
    Z1_list = [j for j in range(L)]
    Z2_list = [j for j in range(L)]
    WX_list = [j for j in range(L)]
    word_list = [j for j in range(L)]

    for m in range(L):
        #文書ごとのオブジェクトの格納用リスト
        WX = np.zeros((d, v[m]), dtype="int32")
        word_data = [i for i in range(d)]
        Z1 = [i for i in range(d)]
        Z2 = [i for i in range(d)]

        for i in range(d):
            #モード変数を生成
            n = w[m][i]
            z1 = np.random.multinomial(1, theta1[i, ], n)
            z1_vec = np.dot(z1, np.arange(L))

            #トピックを生成
            z2 = np.array(rmnom(theta2[i, :, z1_vec], n, k, np.arange(n)).todense())
            z2_vec = np.dot(z2, np.arange(k))

            #単語の生成
            words = np.array(rmnom(phi[m][z2_vec, ], n, v[m], np.arange(n)).todense())
            word_vec = np.dot(words, np.arange(v[m]))

            #文書ごとにデータを格納
            Z1[i] = z1
            Z2[i] = z2
            WX[i, ] = np.sum(words, axis=0)
            word_data[i] = words

        #オブジェクトごとにデータを格納
        WX_list[m] = WX
        word_list[m] = np.array(list(itertools.chain(*[word_data[i] for i in range(d)])))
        Z1_list[m] = np.array(list(itertools.chain(*[Z1[i] for i in range(d)])))
        Z2_list[m] = np.array(list(itertools.chain(*[Z2[i] for i in range(d)])))
    
    #単語がすべて出現していたらbreak
    min_word = np.zeros((L), dtype="int32")
    for j in range(L):
        min_word[j] = np.min(np.sum(WX_list[j], axis=0))

    if np.min(min_word) > 0:
        print(min_word)
        break

1
[2 2 4]


In [6]:
#リストを変換
Z11 = Z1_list[0]; Z12 = Z1_list[1]; Z13 = Z1_list[2]
Z21 = Z2_list[0]; Z22 = Z2_list[1]; Z23 = Z2_list[2]
wd1 = np.dot(word_list[0], np.arange(v1))
wd2 = np.dot(word_list[1], np.arange(v2))
wd3 = np.dot(word_list[2], np.arange(v3))
word_data1 = sparse.csr_matrix(word_list[0]); word_data1_T = word_data1.T
word_data2 = sparse.csr_matrix(word_list[1]); word_data2_T = word_data2.T
word_data3 = sparse.csr_matrix(word_list[2]); word_data3_T = word_data3.T
del Z1_list; del Z2_list; del word_list

In [7]:
####マルコフ連鎖モンテカルロ法でSymCorrLDAを推定####
#トピック尤度と負担率を計算する関数
def LLho(theta, phi, d_id, wd, f, k):
    Lho = theta[d_id, ] * (phi.T)[wd, ]
    topic_rate = Lho / np.sum(Lho, axis=1).reshape(f, 1)
    return Lho, topic_rate

In [8]:
##アルゴリズムの設定
R = 3000   #サンプリング回数
keep = 2   #2回に1回の割合でサンプリング結果を格納
disp = 10
iter = 0
burnin = int(500/keep)

In [9]:
##事前分布の設定
alpha1 = 0.1
alpha2 = 0.1
beta1 = 0.1
beta2 = 0.1
beta3 = 0.1

In [10]:
##パラメータの真値
#モデルパラメータの真値
theta1 = thetat1
theta2 = thetat2
phi1 = phit1
phi2 = phit2
phi3 = phit3
phi = [phi1, phi2, phi3]

#トピックの真値
Zi11 = Z11; Zi12 = Z12; Zi13 = Z13
Zi21 = Z21; Zi22 = Z22; Zi23 = Z23

In [11]:
##パラメータの初期値
#モデルパラメータの初期値
theta1 = np.random.dirichlet(np.repeat(2.0, L), d)
theta2 = np.zeros((d, k, L))
for j in range(L):
    theta2[:, :, j] = np.random.dirichlet(np.repeat(2.0, k), d)
phi1 = np.random.dirichlet(np.repeat(2.0, v1), k)
phi2 = np.random.dirichlet(np.repeat(2.0, v2), k)
phi3 = np.random.dirichlet(np.repeat(2.0, v3), k)

#トピックの初期値
Zi11 = np.array(rmnom(theta1[d_id1, ], f1, L, np.arange(f1)).todense())
Zi12 = np.array(rmnom(theta1[d_id2, ], f2, L, np.arange(f2)).todense())
Zi13 = np.array(rmnom(theta1[d_id3, ], f3, L, np.arange(f3)).todense())
Zi21 = np.array(rmnom(theta2[d_id1, :, 0], f1, k, np.arange(f1)).todense())
Zi22 = np.array(rmnom(theta2[d_id2, :, 1], f2, k, np.arange(f2)).todense())
Zi23 = np.array(rmnom(theta2[d_id3, :, 2], f3, k, np.arange(f3)).todense())

In [12]:
##パラメータの格納用配列
#モデルパラメータの格納用配列
THETA1 = np.zeros((d, L, int(R/keep)))
THETA2 = np.zeros((d, k, L, int(R/keep)))
PHI1 = np.zeros((k, v1, int(R/keep)))
PHI2 = np.zeros((k, v2, int(R/keep)))
PHI3 = np.zeros((k, v3, int(R/keep)))

#トピックの格納用配列
SEG11 = np.zeros((f1, L), dtype="int")
SEG12 = np.zeros((f2, L), dtype="int")
SEG13 = np.zeros((f3, L), dtype="int")
SEG21 = np.zeros((f1, k), dtype="int")
SEG22 = np.zeros((f2, k), dtype="int")
SEG23 = np.zeros((f3, k), dtype="int")

In [13]:
##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst1 = np.sum(np.dot(WX_list[0], np.log(np.sum(WX_list[0], axis=0) / f1)))
LLst2 = np.sum(np.dot(WX_list[1], np.log(np.sum(WX_list[1], axis=0) / f2)))
LLst3 = np.sum(np.dot(WX_list[2], np.log(np.sum(WX_list[2], axis=0) / f3)))
LLst = LLst1 + LLst2 + LLst3
print(np.round(LLst, 3))

#真値での対数尤度
LLbest1 = np.sum(np.log(np.sum(thetat2[d_id1, :, np.dot(Z11, np.arange(L))] * phit1.T[wd1, ], axis=1)))
LLbest2 = np.sum(np.log(np.sum(thetat2[d_id2, :, np.dot(Z12, np.arange(L))] * phit2.T[wd2, ], axis=1)))
LLbest3 = np.sum(np.log(np.sum(thetat2[d_id3, :, np.dot(Z13, np.arange(L))] * phit3.T[wd3, ], axis=1)))
LLbest = LLbest1 + LLbest2 + LLbest3
print(np.round(LLbest, 3))

-7064182.093
-5891857.243


In [14]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##モード変数をサンプリング
    #モードごとの尤度の期待値
    Lho1 = np.zeros((f1, L)); Lho2 = np.zeros((f2, L)); Lho3 = np.zeros((f3, L))
    phi_par1 = phi1.T[wd1, ]; phi_par2 = phi2.T[wd2, ]; phi_par3 = phi3.T[wd3, ]   #モードごとの単語分布
    theta_par1 = theta2[d_id1, :, :]; theta_par2 = theta2[d_id2, :, :]; theta_par3 = theta2[d_id3, :, :]   #モードごとのトピック分布

    for j in range(L):
        Lho1[:, j] = np.dot(theta_par1[:, :, j] * phi_par1, np.repeat(1, k))
        Lho2[:, j] = np.dot(theta_par2[:, :, j] * phi_par2, np.repeat(1, k))
        Lho3[:, j] = np.dot(theta_par3[:, :, j] * phi_par3, np.repeat(1, k))

    #潜在変数zの割当確率
    mode_par1 = theta1[d_id1, ] * Lho1; mode_par2 = theta1[d_id2, ] * Lho2; mode_par3 = theta1[d_id3, ] * Lho3
    z_rate11 = mode_par1 / np.dot(mode_par1, np.repeat(1, L)).reshape(f1, 1)
    z_rate12 = mode_par2 / np.dot(mode_par2, np.repeat(1, L)).reshape(f2, 1)
    z_rate13 = mode_par3 / np.dot(mode_par3, np.repeat(1, L)).reshape(f3, 1)

    #多項分布よりモード変数をサンプリング
    Sparse_Zi11 = rmnom(z_rate11, f1, L, np.arange(f1)).tocsr()
    Sparse_Zi12 = rmnom(z_rate12, f2, L, np.arange(f2)).tocsr()
    Sparse_Zi13 = rmnom(z_rate13, f3, L, np.arange(f3)).tocsr()
    Zi11 = np.array(Sparse_Zi11.todense()); Zi12 = np.array(Sparse_Zi12.todense()); Zi13 = np.array(Sparse_Zi13.todense())


    ##トピックをサンプリング
    #モードごとのトピック分布の尤度
    topic_par1 = theta2[d_id1, :, np.dot(Zi11, np.arange(L))] * phi_par1
    topic_par2 = theta2[d_id2, :, np.dot(Zi12, np.arange(L))] * phi_par2
    topic_par3 = theta2[d_id3, :, np.dot(Zi13, np.arange(L))] * phi_par3

    #潜在変数zの割当確率
    z_rate21 = topic_par1 / np.dot(topic_par1, np.repeat(1, k)).reshape(f1, 1)
    z_rate22 = topic_par2 / np.dot(topic_par2, np.repeat(1, k)).reshape(f2, 1)
    z_rate23 = topic_par3 / np.dot(topic_par3, np.repeat(1, k)).reshape(f3, 1)

    #多項分布よりトピックをサンプリング
    Sparse_Zi21 = rmnom(z_rate21, f1, k, np.arange(f1)).tocsr()
    Sparse_Zi22 = rmnom(z_rate22, f2, k, np.arange(f2)).tocsr()
    Sparse_Zi23 = rmnom(z_rate23, f3, k, np.arange(f3)).tocsr()
    Zi21 = np.array(Sparse_Zi21.todense()); Zi22 = np.array(Sparse_Zi22.todense()); Zi23 = np.array(Sparse_Zi23.todense())


    ##モード分布のパラメータをサンプリング
    #ディリクレ分布のパラメータ
    ysum1 = np.array(np.dot(d_dt1, Sparse_Zi11).todense())
    ysum2 = np.array(np.dot(d_dt2, Sparse_Zi12).todense())
    ysum3 = np.array(np.dot(d_dt3, Sparse_Zi13).todense())
    ysum = ysum1 + ysum2 + ysum3 + alpha1

    #ディリクレ分布からパタメータをサンプリング
    for i in range(d):
        theta1[i, ] = np.random.dirichlet(ysum[i, ], 1)

    ##トピック分布のパラメータをサンプリング
    for j in range(L):
        #モード変数ごとのトピック割当
        Zi1_T = (Zi21 * Zi11[:, m].reshape(f1, 1)).T
        Zi2_T = (Zi22 * Zi12[:, m].reshape(f2, 1)).T
        Zi3_T = (Zi23 * Zi13[:, m].reshape(f3, 1)).T

        for i in range(d):
            #ディリクレ分布からパラメータをサンプリング
            wsum1 = np.dot(Zi1_T[:, d_list[0][i]], d_vec[0][i])
            wsum2 = np.dot(Zi2_T[:, d_list[1][i]], d_vec[1][i])
            wsum3 = np.dot(Zi3_T[:, d_list[2][i]], d_vec[2][i])
            wsum = wsum1 + wsum2 + wsum3 + alpha2
            theta2[i, :, m] = np.random.dirichlet(wsum, 1)

            
    ##単語分布のパラメータをサンプリング
    #ディリクレ分布のパラメータ
    vsum1 = np.array(np.dot(Sparse_Zi21.T, word_data1).todense()) + beta1
    vsum2 = np.array(np.dot(Sparse_Zi22.T, word_data2).todense()) + beta2
    vsum3 = np.array(np.dot(Sparse_Zi23.T, word_data3).todense()) + beta3

    #ディリクレ分布からパラメータをサンプリング
    for j in range(k):
        phi1[j, :] = np.random.dirichlet(vsum1[j, ], 1)
        phi2[j, :] = np.random.dirichlet(vsum2[j, ], 1)
        phi3[j, :] = np.random.dirichlet(vsum3[j, ], 1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        THETA1[:, :, mkeep] = theta1
        THETA2[:, :, :, mkeep] = theta2
        PHI1[:, :, mkeep] = phi1
        PHI2[:, :, mkeep] = phi2
        PHI3[:, :, mkeep] = phi3
        
    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        SEG11 = SEG11 + Zi11
        SEG12 = SEG12 + Zi12
        SEG13 = SEG13 + Zi13
        SEG21 = SEG21 + Zi21
        SEG22 = SEG22 + Zi22
        SEG23 = SEG23 + Zi23
        
    if rp%disp==0:
        #対数尤度の更新
        LL1 = np.sum(np.log(np.sum(topic_par1, axis=1)))
        LL2 = np.sum(np.log(np.sum(topic_par2, axis=1)))
        LL3 = np.sum(np.log(np.sum(topic_par3, axis=1)))
        LL = LL1 + LL2 + LL3

        #サンプリング結果を確認
        print(rp)
        print(np.round(np.array([LL1, LLst1, LLbest1]), 1))
        print(np.round(np.array([LL2, LLst2, LLbest2]), 1))
        print(np.round(np.array([LL3, LLst3, LLbest3]), 1))
        print(np.round(np.array([LL, LLst, LLbest]), 1))

0
[-2934259.8 -2701190.6 -2270118.1]
[-2476292.7 -2278780.5 -1897956.6]
[-2267631.4 -2084210.9 -1723782.5]
[-7678183.9 -7064182.1 -5891857.2]
10
[-2685128.  -2701190.6 -2270118.1]
[-2264492.2 -2278780.5 -1897956.6]
[-2070537.6 -2084210.9 -1723782.5]
[-7020157.7 -7064182.1 -5891857.2]
20
[-2545617.2 -2701190.6 -2270118.1]
[-2139440.4 -2278780.5 -1897956.6]
[-1951875.  -2084210.9 -1723782.5]
[-6636932.5 -7064182.1 -5891857.2]
30
[-2444348.1 -2701190.6 -2270118.1]
[-2050583.1 -2278780.5 -1897956.6]
[-1868098.9 -2084210.9 -1723782.5]
[-6363030.1 -7064182.1 -5891857.2]
40
[-2410615.  -2701190.6 -2270118.1]
[-2022004.4 -2278780.5 -1897956.6]
[-1842053.4 -2084210.9 -1723782.5]
[-6274672.8 -7064182.1 -5891857.2]
50
[-2396247.9 -2701190.6 -2270118.1]
[-2010021.8 -2278780.5 -1897956.6]
[-1829557.9 -2084210.9 -1723782.5]
[-6235827.5 -7064182.1 -5891857.2]
60
[-2385208.1 -2701190.6 -2270118.1]
[-2000186.7 -2278780.5 -1897956.6]
[-1821397.3 -2084210.9 -1723782.5]
[-6206792.2 -7064182.1 -5891857.2]


570
[-2363888.2 -2701190.6 -2270118.1]
[-1980481.8 -2278780.5 -1897956.6]
[-1801789.7 -2084210.9 -1723782.5]
[-6146159.7 -7064182.1 -5891857.2]
580
[-2364271.4 -2701190.6 -2270118.1]
[-1980074.7 -2278780.5 -1897956.6]
[-1802144.1 -2084210.9 -1723782.5]
[-6146490.2 -7064182.1 -5891857.2]
590
[-2364038.5 -2701190.6 -2270118.1]
[-1980787.5 -2278780.5 -1897956.6]
[-1801837.3 -2084210.9 -1723782.5]
[-6146663.4 -7064182.1 -5891857.2]
600
[-2364073.8 -2701190.6 -2270118.1]
[-1980966.5 -2278780.5 -1897956.6]
[-1801781.1 -2084210.9 -1723782.5]
[-6146821.5 -7064182.1 -5891857.2]
610
[-2364141.1 -2701190.6 -2270118.1]
[-1980663.3 -2278780.5 -1897956.6]
[-1801568.5 -2084210.9 -1723782.5]
[-6146373.  -7064182.1 -5891857.2]
620
[-2363914.5 -2701190.6 -2270118.1]
[-1980583.3 -2278780.5 -1897956.6]
[-1802001.3 -2084210.9 -1723782.5]
[-6146499.1 -7064182.1 -5891857.2]
630
[-2364120.7 -2701190.6 -2270118.1]
[-1980617.3 -2278780.5 -1897956.6]
[-1801642.9 -2084210.9 -1723782.5]
[-6146380.9 -7064182.1 -589

1140
[-2363788.4 -2701190.6 -2270118.1]
[-1980813.4 -2278780.5 -1897956.6]
[-1801813.2 -2084210.9 -1723782.5]
[-6146415.  -7064182.1 -5891857.2]
1150
[-2363856.8 -2701190.6 -2270118.1]
[-1980663.8 -2278780.5 -1897956.6]
[-1801950.7 -2084210.9 -1723782.5]
[-6146471.3 -7064182.1 -5891857.2]
1160
[-2363882.7 -2701190.6 -2270118.1]
[-1980712.7 -2278780.5 -1897956.6]
[-1801923.6 -2084210.9 -1723782.5]
[-6146519.  -7064182.1 -5891857.2]
1170
[-2363762.4 -2701190.6 -2270118.1]
[-1980863.7 -2278780.5 -1897956.6]
[-1802116.  -2084210.9 -1723782.5]
[-6146742.1 -7064182.1 -5891857.2]
1180
[-2363978.7 -2701190.6 -2270118.1]
[-1980883.7 -2278780.5 -1897956.6]
[-1802171.1 -2084210.9 -1723782.5]
[-6147033.5 -7064182.1 -5891857.2]
1190
[-2364492.1 -2701190.6 -2270118.1]
[-1980724.9 -2278780.5 -1897956.6]
[-1802048.3 -2084210.9 -1723782.5]
[-6147265.3 -7064182.1 -5891857.2]
1200
[-2364260.5 -2701190.6 -2270118.1]
[-1980857.4 -2278780.5 -1897956.6]
[-1802132.8 -2084210.9 -1723782.5]
[-6147250.8 -7064182

1710
[-2364307.2 -2701190.6 -2270118.1]
[-1980599.7 -2278780.5 -1897956.6]
[-1801972.1 -2084210.9 -1723782.5]
[-6146879.  -7064182.1 -5891857.2]
1720
[-2364213.3 -2701190.6 -2270118.1]
[-1980683.6 -2278780.5 -1897956.6]
[-1801802.1 -2084210.9 -1723782.5]
[-6146699.  -7064182.1 -5891857.2]
1730
[-2364599.4 -2701190.6 -2270118.1]
[-1980741.2 -2278780.5 -1897956.6]
[-1802070.7 -2084210.9 -1723782.5]
[-6147411.3 -7064182.1 -5891857.2]
1740
[-2364330.  -2701190.6 -2270118.1]
[-1980984.5 -2278780.5 -1897956.6]
[-1802167.8 -2084210.9 -1723782.5]
[-6147482.3 -7064182.1 -5891857.2]
1750
[-2364232.9 -2701190.6 -2270118.1]
[-1980885.7 -2278780.5 -1897956.6]
[-1802020.9 -2084210.9 -1723782.5]
[-6147139.5 -7064182.1 -5891857.2]
1760
[-2364444.2 -2701190.6 -2270118.1]
[-1981096.5 -2278780.5 -1897956.6]
[-1802046.  -2084210.9 -1723782.5]
[-6147586.7 -7064182.1 -5891857.2]
1770
[-2364696.7 -2701190.6 -2270118.1]
[-1980912.9 -2278780.5 -1897956.6]
[-1802289.6 -2084210.9 -1723782.5]
[-6147899.1 -7064182

2280
[-2363921.5 -2701190.6 -2270118.1]
[-1980872.5 -2278780.5 -1897956.6]
[-1802045.2 -2084210.9 -1723782.5]
[-6146839.2 -7064182.1 -5891857.2]
2290
[-2364071.6 -2701190.6 -2270118.1]
[-1980605.8 -2278780.5 -1897956.6]
[-1801984.8 -2084210.9 -1723782.5]
[-6146662.2 -7064182.1 -5891857.2]
2300
[-2364248.4 -2701190.6 -2270118.1]
[-1980606.4 -2278780.5 -1897956.6]
[-1801827.  -2084210.9 -1723782.5]
[-6146681.9 -7064182.1 -5891857.2]
2310
[-2364635.5 -2701190.6 -2270118.1]
[-1981170.8 -2278780.5 -1897956.6]
[-1802210.9 -2084210.9 -1723782.5]
[-6148017.3 -7064182.1 -5891857.2]
2320
[-2364147.  -2701190.6 -2270118.1]
[-1980838.2 -2278780.5 -1897956.6]
[-1802233.1 -2084210.9 -1723782.5]
[-6147218.3 -7064182.1 -5891857.2]
2330
[-2364309.4 -2701190.6 -2270118.1]
[-1980969.2 -2278780.5 -1897956.6]
[-1801941.5 -2084210.9 -1723782.5]
[-6147220.1 -7064182.1 -5891857.2]
2340
[-2364136.9 -2701190.6 -2270118.1]
[-1980877.3 -2278780.5 -1897956.6]
[-1801984.3 -2084210.9 -1723782.5]
[-6146998.5 -7064182

2850
[-2363931.6 -2701190.6 -2270118.1]
[-1980516.4 -2278780.5 -1897956.6]
[-1801784.  -2084210.9 -1723782.5]
[-6146232.  -7064182.1 -5891857.2]
2860
[-2363849.  -2701190.6 -2270118.1]
[-1980343.7 -2278780.5 -1897956.6]
[-1801670.3 -2084210.9 -1723782.5]
[-6145863.  -7064182.1 -5891857.2]
2870
[-2364398.7 -2701190.6 -2270118.1]
[-1980433.9 -2278780.5 -1897956.6]
[-1801976.6 -2084210.9 -1723782.5]
[-6146809.2 -7064182.1 -5891857.2]
2880
[-2364085.6 -2701190.6 -2270118.1]
[-1980585.7 -2278780.5 -1897956.6]
[-1802069.4 -2084210.9 -1723782.5]
[-6146740.6 -7064182.1 -5891857.2]
2890
[-2363810.9 -2701190.6 -2270118.1]
[-1980310.  -2278780.5 -1897956.6]
[-1801678.6 -2084210.9 -1723782.5]
[-6145799.5 -7064182.1 -5891857.2]
2900
[-2363995.8 -2701190.6 -2270118.1]
[-1980541.6 -2278780.5 -1897956.6]
[-1801634.8 -2084210.9 -1723782.5]
[-6146172.1 -7064182.1 -5891857.2]
2910
[-2363914.3 -2701190.6 -2270118.1]
[-1980672.7 -2278780.5 -1897956.6]
[-1801791.7 -2084210.9 -1723782.5]
[-6146378.6 -7064182